#Problem

Suppose that you have 3 machines to manufacture shoes and the cost of each machine is :

C1= 0.1*n1^2 +2*n1

C2= 6*n2

c3=7*n3


where Ci is cost for production  of machine i ,

ni is the number of shows manufactured in machine

each machine has a limit of production of 1000 shoes

for a total production of 2100 shoes, how many shoes should each machine made in order to minimize the total cost?

#Solution

objective Function : min C1+C2+C3

Constraints:

n1+n2+n3= 2100

C1= 0.1n1^2 +2n1

C2= 6n2

c3=7n3

0 <= n1,n2,n3 <=1000

we can use non-linear solver here but we can also use linear solver like gurobi

#installation of pyomo

In [ ]:
 !pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [ ]:
!pip install pyomo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.8 MB/s eta 0:00:00


In [ ]:
!conda install -c conda-forge glpk

Solving environment: / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - glpk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boltons-23.0.0             |     pyhd8ed1ab_0         296 KB  conda-forge
    ca-certificates-2023.5.7   |       hbcca054_0         145 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    conda-23.3.1               |  py310hff52083_0         941 KB  conda-forge
    glpk-5.0                   |       h445213a_0        1023 KB  conda-forge
    gmp-6.2.1                  |       h58526e2_0         806 KB  conda-forge
    jsonpatch-1.32             |     pyhd8ed1ab_0          14 KB  conda-forge
    jsonpointer-2.0            |             py_0           9 KB  conda-forge
    openssl-3.1.1              |       hd590300_1         2.5 MB  conda-for

#Solver

Need Solver couenne.....Cause gurobi will work...glpk a

In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

#Code

In [ ]:
import pyomo.environ as pyo, numpy as np
from pyomo.environ import *
from pyomo.opt import SolverFactory

#creation of the model and variables
model = pyo.ConcreteModel()
model.C = pyo.Var(range(1,4)) #here C is the machine's cost
model.n = pyo.Var(range(1,4), within=Integers, bounds=(0,1000)) #n number of shoes #care about the sequence

C = model.C
n = model.n

#objective function
model.obj = pyo.Objective(expr = pyo.summation(C)) #or (expr= C[1]+C[2]+C[3])

#constraint
model.total = pyo.Constraint(expr = pyo.summation(n) == 2100) # (expr =  n[1]+n[2]+n[3]==2100)
model.C1 = pyo.Constraint(expr = C[1] == 0.01*n[1]*n[1] + 2*n[1])
model.C2 = pyo.Constraint(expr = C[2] == 6*n[2])
model.C3 = pyo.Constraint(expr = C[3] == 7*n[3])

#solve
opt = SolverFactory('couenne', executable= '/content/bin/couenne') #gurobi is better,  glpk & cbc won't  work
opt.solve(model)

#print
print('n1', pyo.value(n[1]))
print('n2', pyo.value(n[2]))
print('n3', pyo.value(n[3]))
print('nTotal', pyo.value(pyo.summation(n)))

print('C1', pyo.value(C[1]))
print('C2', pyo.value(C[2]))
print('C3', pyo.value(C[3]))
print('CTotal', pyo.value(pyo.summation(C)))



n1 250.0
n2 1000.0
n3 850.0
nTotal 2100.0
C1 1125.0
C2 6000.0
C3 5950.0
CTotal 13075.0
